# Imports

In [26]:
import pandas as pd
pd.set_option('display.max_columns', None)

import os
import http.client
from datetime import datetime, timedelta
import json
import time
import traceback

# Variables

In [27]:
league = 'Turkey_SuperLeague'
footballdata_wIndexes_path = os.path.abspath(os.getcwd())+"\\Java\\files\\"+league+".csv"
idx_mappings_dir = os.path.abspath(os.getcwd())+"\\Data\\footballAPI_idx_mappings\\"+league+".csv"

footballdata_wTeamStats_outputPath = \
    os.path.abspath(os.getcwd())+"\\Data\\teamStats_byLeague\\footballdata_wTeamStats_"+league+".csv"

api_dailyLimit = 10000

# Load Football-Data File with Indexes

In [28]:
footballdata_wIndexes = pd.read_csv(footballdata_wIndexes_path, delimiter=";", index_col=False)
footballdata_wIndexes = footballdata_wIndexes.loc[:, ~footballdata_wIndexes.columns.str.contains('^Unnamed')]
footballdata_wIndexes

,Season,Date,HomeTeamid,HomeTeamName,AwayTeamId,AwayTeamName,HomeGoals,AwayGoals,FullTimeResult,AverageHomeWinOdd,AverageDrawWinOdd,AverageAwayWinOdd,AvgOver2_5,AvgUnder2_5,Div
0,2010,14/08/2010,1003,Bucaspor,549,Besiktas,0,1,A,6.00,3.80,1.57,1.87,1.87,203
1,2010,14/08/2010,3572,Eskisehirspor,997,Genclerbirligi,0,0,D,2.00,3.40,3.75,2.03,1.73,203
2,2010,14/08/2010,1002,Sivasspor,645,Galatasaray,2,1,H,5.25,3.60,1.67,1.92,1.82,203
3,2010,15/08/2010,1010,Ankaragucu,998,Trabzonspor,0,2,A,3.75,3.40,2.00,2.03,1.73,203
4,2010,15/08/2010,611,Fenerbahce,1005,Antalyaspor,4,0,H,1.44,4.33,7.00,1.71,2.02,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,2022,13/08/2022,3563,Ad. Demirspor,1002,Sivasspor,3,0,H,1.94,3.64,3.73,1.83,1.97,203
3806,2022,13/08/2022,645,Galatasaray,3574,Giresunspor,0,1,A,1.43,4.59,7.10,1.72,2.10,203
3807,2022,14/08/2022,3573,Gaziantep,1010,Ankaragucu,1,0,H,1.99,3.51,3.66,1.89,1.90,203
3808,2022,14/08/2022,3577,Umraniyespor,1005,Antalyaspor,0,1,A,2.99,3.41,2.31,1.82,1.98,203


In [29]:
footballdata_wIndexes.count()

Season               3810
Date                 3810
HomeTeamid           3810
HomeTeamName         3810
AwayTeamId           3810
AwayTeamName         3810
HomeGoals            3810
AwayGoals            3810
FullTimeResult       3810
AverageHomeWinOdd    3802
AverageDrawWinOdd    3802
AverageAwayWinOdd    3802
AvgOver2_5           3809
AvgUnder2_5          3809
Div                  3810
dtype: int64

## Store Index Mapping

In [30]:
idxs = footballdata_wIndexes[['HomeTeamName','HomeTeamid','Div']].drop_duplicates().set_index('HomeTeamName')\
    .sort_values('HomeTeamName')
idxs

,HomeTeamid,Div
HomeTeamName,,
Ad. Demirspor,3563,203
Adanaspor,3564,203
Akhisar Belediyespor,995,203
Alanyaspor,996,203
Altay,3566,203
Ankaragucu,1010,203
Antalyaspor,1005,203
Balikesirspor,3568,203
Besiktas,549,203


In [31]:
idxs.to_csv(idx_mappings_dir)

# Call API to get match Stats

In [32]:
keys_columns = [(['form'], 'form'),
        
        (['fixtures','wins','home'], 'home_wins'),
        (['fixtures','draws','home'], 'home_draws'),
        (['fixtures','loses','home'], 'home_loses'),
        (['fixtures','wins','away'], 'away_wins'),
        (['fixtures','draws','away'], 'away_draws'),
        (['fixtures','loses','away'], 'away_loses'),
       
       (['goals','for','total','home'], 'home_goals_for'),
       (['goals','for','total','away'], 'away_goals_for'),
       (['goals','for','total','total'], 'total_goals_for'),
       (['goals','for','average','home'], 'average_home_goals_for'),
       (['goals','for','average','away'], 'average_away_goals_for'),
       (['goals','for','average','total'], 'average_total_goals_for'),
       (['goals','for','minute','0-15','total'], 'goals_for_minute_0_15'),
       (['goals','for','minute','0-15','percentage'], 'goals_for_minute_0_15_percentage'),
       (['goals','for','minute','16-30','total'], 'goals_for_minute_16_30'),
       (['goals','for','minute','16-30','percentage'], 'goals_for_minute_16_30_percentage'),
       (['goals','for','minute','31-45','total'], 'goals_for_minute_31_45'),
       (['goals','for','minute','31-45','percentage'], 'goals_for_minute_31_45_percentage'),
       (['goals','for','minute','46-60','total'], 'goals_for_minute_46_60'),
       (['goals','for','minute','46-60','percentage'], 'goals_for_minute_46_60_percentage'),
       (['goals','for','minute','61-75','total'], 'goals_for_minute_61_75'),
       (['goals','for','minute','61-75','percentage'], 'goals_for_minute_61_75_percentage'),
       (['goals','for','minute','76-90','total'], 'goals_for_minute_76_90'),
       (['goals','for','minute','76-90','percentage'], 'goals_for_minute_76_90_percentage'),
       (['goals','for','minute','91-105','total'], 'goals_for_minute_91_105'),
       (['goals','for','minute','91-105','percentage'], 'goals_for_minute_91_105_percentage'),
        
       (['goals','against','total','home'], 'home_goals_against'),
       (['goals','against','total','away'], 'away_goals_against'),
       (['goals','against','total','total'], 'total_goals_against'),
       (['goals','against','average','home'], 'average_home_goals_against'),
       (['goals','against','average','away'], 'average_away_goals_against'),
       (['goals','against','average','total'], 'average_total_goals_against'),
       (['goals','against','minute','0-15','total'], 'goals_against_minute_0_15'),
       (['goals','against','minute','0-15','percentage'], 'goals_against_minute_0_15_percentage'),
       (['goals','against','minute','16-30','total'], 'goals_against_minute_16_30'),
       (['goals','against','minute','16-30','percentage'], 'goals_against_minute_16_30_percentage'),
       (['goals','against','minute','31-45','total'], 'goals_against_minute_31_45'),
       (['goals','against','minute','31-45','percentage'], 'goals_against_minute_31_45_percentage'),
       (['goals','against','minute','46-60','total'], 'goals_against_minute_46_60'),
       (['goals','against','minute','46-60','percentage'], 'goals_against_minute_46_60_percentage'),
       (['goals','against','minute','61-75','total'], 'goals_against_minute_61_75'),
       (['goals','against','minute','61-75','percentage'], 'goals_against_minute_61_75_percentage'),
       (['goals','against','minute','76-90','total'], 'goals_against_minute_76_90'),
       (['goals','against','minute','76-90','percentage'], 'goals_against_minute_76_90_percentage'),
       (['goals','against','minute','91-105','total'], 'goals_against_minute_91_105'),
       (['goals','against','minute','91-105','percentage'], 'goals_against_minute_91_105_percentage'),
        
       (['biggest','streak','wins'], 'biggest_win_streak'),
       (['biggest','streak','draws'], 'biggest_draw_streak'),
       (['biggest','streak','loses'], 'biggest_lose_streak'),
        
       (['biggest','wins','home'], 'biggest_home_win'),
       (['biggest','wins','away'], 'biggest_away_win'),
       (['biggest','loses','home'], 'biggest_home_lose'),
       (['biggest','loses','away'], 'biggest_away_lose'),
       (['biggest','goals','for','home'], 'biggest_home_goals_for'),
       (['biggest','goals','for','away'], 'biggest_away_goals_for'),
       (['biggest','goals','against','home'], 'biggest_home_goals_against'),
       (['biggest','goals','against','away'], 'biggest_away_goals_against'),
        
       (['clean_sheet','home'], 'home_clean_sheets'),
       (['clean_sheet','away'], 'away_clean_sheets'),
       (['clean_sheet','total'], 'total_clean_sheets'),
        
       (['failed_to_score','home'], 'failed_to_score_home'),
       (['failed_to_score','away'], 'failed_to_score_away'),
       (['failed_to_score','total'], 'failed_to_score_total'),
        
       (['penalty','scored','total'], 'penalties_scored'),
       (['penalty','scored','percentage'], 'penalties_scored_percentage'),
       (['penalty','missed','total'], 'penalties_missed'),
       (['penalty','missed','percentage'], 'penalties_missed_percentage'),
       (['penalty','total'], 'penalties_for'),
        
        (['lineups'], 'lineups'),
        
       (['cards','yellow','0-15','total'], 'yellow_cards_minute_0_15'),
       (['cards','yellow','0-15','percentage'], 'yellow_cards_minute_0_15_percentage'),
       (['cards','yellow','16-30','total'], 'yellow_cards_minute_16_30'),
       (['cards','yellow','16-30','percentage'], 'yellow_cards_minute_16_30_percentage'),
       (['cards','yellow','31-45','total'], 'yellow_cards_minute_31_45'),
       (['cards','yellow','31-45','percentage'], 'yellow_cards_minute_31_45_percentage'),
       (['cards','yellow','46-60','total'], 'yellow_cards_minute_46_60'),
       (['cards','yellow','46-60','percentage'], 'yellow_cards_minute_46_60_percentage'),
       (['cards','yellow','61-75','total'], 'yellow_cards_minute_61_75'),
       (['cards','yellow','61-75','percentage'], 'yellow_cards_minute_61_75_percentage'),
       (['cards','yellow','76-90','total'], 'yellow_cards_minute_76_90'),
       (['cards','yellow','76-90','percentage'], 'yellow_cards_minute_76_90_percentage'),
       (['cards','yellow','91-105','total'], 'yellow_cards_minute_91_105'),
       (['cards','yellow','91-105','percentage'], 'yellow_cards_minute_91_105_percentage'),
        
       (['cards','red','0-15','total'], 'red_cards_minute_0_15'),
       (['cards','red','0-15','percentage'], 'red_cards_minute_0_15_percentage'),
       (['cards','red','16-30','total'], 'red_cards_minute_16_30'),
       (['cards','red','16-30','percentage'], 'red_cards_minute_16_30_percentage'),
       (['cards','red','31-45','total'], 'red_cards_minute_31_45'),
       (['cards','red','31-45','percentage'], 'red_cards_minute_31_45_percentage'),
       (['cards','red','46-60','total'], 'red_cards_minute_46_60'),
       (['cards','red','46-60','percentage'], 'red_cards_minute_46_60_percentage'),
       (['cards','red','61-75','total'], 'red_cards_minute_61_75'),
       (['cards','red','61-75','percentage'], 'red_cards_minute_61_75_percentage'),
       (['cards','red','76-90','total'], 'red_cards_minute_76_90'),
       (['cards','red','76-90','percentage'], 'red_cards_minute_76_90_percentage'),
       (['cards','red','91-105','total'], 'red_cards_minute_91_105'),
       (['cards','red','91-105','percentage'], 'red_cards_minute_91_105_percentage')]

In [34]:
try:
    footballdata_wTeamStats = pd.read_csv(footballdata_wTeamStats_outputPath, index_col=0)
except FileNotFoundError:
    footballdata_wTeamStats = pd.DataFrame()

start_idx = len(footballdata_wTeamStats)
print(start_idx)

count = 0
for index, row in footballdata_wIndexes[start_idx:].iterrows():
    
    try:
        
        # subtract 1 day to get stats prior to the match
        date = (datetime.strptime(row['Date'], '%d/%m/%Y')-timedelta(days=1)).strftime("%Y-%m-%d")
        home_id = row['HomeTeamid']
        away_id = row['AwayTeamId']
        league_id = row['Div']
        season = row['Season']

        # api connection
        conn = http.client.HTTPSConnection("v3.football.api-sports.io")
        headers = {
            'x-rapidapi-host': "v3.football.api-sports.io",
            'x-rapidapi-key': "15f514ec0209e0f49397240d5e166bd9"
            }
        data = []

        # home team
        conn.request("GET",
             f"https://v3.football.api-sports.io/teams/statistics?league={league_id}&season={season}&team={home_id}&date={date}",
             headers=headers)
        res = conn.getresponse()
        data = res.read()
        home_data = json.loads(data.decode("utf-8"))
        count+=1

        # away team
        conn.request("GET",
             f"https://v3.football.api-sports.io/teams/statistics?league={league_id}&season={season}&team={away_id}&date={date}",
             headers=headers)
        res = conn.getresponse()
        data = res.read()
        away_data = json.loads(data.decode("utf-8"))
        count+=1

        values = []
        columns = []
        for i in range(2):
            if i == 0:
                team_stats = home_data
                header_ini = 'home_'
            elif i == 1:
                team_stats = away_data
                header_ini = 'away_'

            for (keys, column) in keys_columns:
                value = team_stats['response']
                for key in keys:
                    value = value[key]
                columns.append(header_ini+column)
                values.append(value)

        row = row.append(pd.Series(values, index=columns))

        footballdata_wTeamStats = footballdata_wTeamStats.append(row, ignore_index=True)

        # don't exceed api daily limit
        if count >= api_dailyLimit:
            break
        elif count%1000 == 0:
            print(f"We're here: Daily Requests:{int(count)}, Table Size:{index}")

        time.sleep(0.4)

    except Exception as e:
        print(home_data)
        print(away_data)
        print(e)
        print(traceback.format_exc())
        print(f"We stopped here: {int(count)}, {index}")
        break
        
footballdata_wTeamStats.to_csv(footballdata_wTeamStats_outputPath)
footballdata_wTeamStats

279
We're here: Daily Requests:1000, Table Size:778
We're here: Daily Requests:2000, Table Size:1278
We're here: Daily Requests:3000, Table Size:1778
We're here: Daily Requests:4000, Table Size:2278
We're here: Daily Requests:5000, Table Size:2778
We're here: Daily Requests:6000, Table Size:3278
We're here: Daily Requests:7000, Table Size:3778


,AverageAwayWinOdd,AverageDrawWinOdd,AverageHomeWinOdd,AvgOver2_5,AvgUnder2_5,AwayGoals,AwayTeamId,AwayTeamName,Date,Div,FullTimeResult,HomeGoals,HomeTeamName,HomeTeamid,Season,away_average_away_goals_against,away_average_away_goals_for,away_average_home_goals_against,away_average_home_goals_for,away_average_total_goals_against,away_average_total_goals_for,away_away_clean_sheets,away_away_draws,away_away_goals_against,away_away_goals_for,away_away_loses,away_away_wins,away_biggest_away_goals_against,away_biggest_away_goals_for,away_biggest_away_lose,away_biggest_away_win,away_biggest_draw_streak,away_biggest_home_goals_against,away_biggest_home_goals_for,away_biggest_home_lose,away_biggest_home_win,away_biggest_lose_streak,away_biggest_win_streak,away_failed_to_score_away,away_failed_to_score_home,away_failed_to_score_total,away_form,away_goals_against_minute_0_15,away_goals_against_minute_0_15_percentage,away_goals_against_minute_16_30,away_goals_against_minute_16_30_percentage,away_goals_against_minute_31_45,away_goals_against_minute_31_45_percentage,away_goals_against_minute_46_60,away_goals_against_minute_46_60_percentage,away_goals_against_minute_61_75,away_goals_against_minute_61_75_percentage,away_goals_against_minute_76_90,away_goals_against_minute_76_90_percentage,away_goals_against_minute_91_105,away_goals_against_minute_91_105_percentage,away_goals_for_minute_0_15,away_goals_for_minute_0_15_percentage,away_goals_for_minute_16_30,away_goals_for_minute_16_30_percentage,away_goals_for_minute_31_45,away_goals_for_minute_31_45_percentage,away_goals_for_minute_46_60,away_goals_for_minute_46_60_percentage,away_goals_for_minute_61_75,away_goals_for_minute_61_75_percentage,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_goals_for_minute_91_105,away_goals_for_minute_91_105_percentage,away_home_clean_sheets,away_home_draws,away_home_goals_against,away_home_goals_for,away_home_loses,away_home_wins,away_lineups,away_penalties_for,away_penalties_missed,away_penalties_missed_percentage,away_penalties_scored,away_penalties_scored_percentage,away_red_cards_minute_0_15,away_red_cards_minute_0_15_percentage,away_red_cards_minute_16_30,away_red_cards_minute_16_30_percentage,away_red_cards_minute_31_45,away_red_cards_minute_31_45_percentage,away_red_cards_minute_46_60,away_red_cards_minute_46_60_percentage,away_red_cards_minute_61_75,away_red_cards_minute_61_75_percentage,away_red_cards_minute_76_90,away_red_cards_minute_76_90_percentage,away_red_cards_minute_91_105,away_red_cards_minute_91_105_percentage,away_total_clean_sheets,away_total_goals_against,away_total_goals_for,away_yellow_cards_minute_0_15,away_yellow_cards_minute_0_15_percentage,away_yellow_cards_minute_16_30,away_yellow_cards_minute_16_30_percentage,away_yellow_cards_minute_31_45,away_yellow_cards_minute_31_45_percentage,away_yellow_cards_minute_46_60,away_yellow_cards_minute_46_60_percentage,away_yellow_cards_minute_61_75,away_yellow_cards_minute_61_75_percentage,away_yellow_cards_minute_76_90,away_yellow_cards_minute_76_90_percentage,away_yellow_cards_minute_91_105,away_yellow_cards_minute_91_105_percentage,home_average_away_goals_against,home_average_away_goals_for,home_average_home_goals_against,home_average_home_goals_for,home_average_total_goals_against,home_average_total_goals_for,home_away_clean_sheets,home_away_draws,home_away_goals_against,home_away_goals_for,home_away_loses,home_away_wins,home_biggest_away_goals_against,home_biggest_away_goals_for,home_biggest_away_lose,home_biggest_away_win,home_biggest_draw_streak,home_biggest_home_goals_against,home_biggest_home_goals_for,home_biggest_home_lose,home_biggest_home_win,home_biggest_lose_streak,home_biggest_win_streak,home_failed_to_score_away,home_failed_to_score_home,home_failed_to_score_total,home_form,home_goals_against_minute_0_15,home_goals_against_minute_0_15_percentage,home_goals_against_minute_16_30,home_goals_against_minute_16_30_percentage,home_goals_against_minute_31_45,home_